# Seatle and Boston Airbnb Open Data
## Data Acquisition notebook

**Author:** Paola Rocha  
**Date:** March 2nd, 2024

**Description**  
The objective of this notebook is to prepare and clean data for [Boston](https://www.kaggle.com/datasets/airbnb/boston?select=calendar.csv) and [Seatle](https://www.kaggle.com/datasets/airbnb/seattle?resource=download) Airbnb Open Data. Tambièn, este notebook sirve para darle un primer vistazo a los datos que cuenta el dataset y familiarizarnos con los datos.

**Notebook contents**
1. Libraries
2. Dataset description
3. Data acquistion
4. Cleaning data
5. Saving data

### 1. Libraries

In [1]:
# Data processing
import pandas as pd
import numpy as np

### 2. Dataset description

The datasets used in this projects, consist of activity listing in [Boston](https://www.kaggle.com/datasets/airbnb/boston?select=calendar.csv) and [Seatle](https://www.kaggle.com/datasets/airbnb/seattle?resource=download) Airbnb. The data can be found and downloaded in Kaggle. Both datasets consists of 3 CSV files:
- Listing: contains average review score. Each id (`listing_id`) is an accommodation registered in Airbnb.
- Reviews: review comments made by users.
- Calendar: contains the price and availability in a year.

All these 3 datasets can be merged using `listing_id`.

### 3. Data acquisition

#### Seatle data
Using pandas library to read Seatle data

In [2]:
seatle_listings = pd.read_csv('../data/Seatle/listings.csv')
seatle_reviews = pd.read_csv('../data/Seatle/reviews.csv', parse_dates=[2])
seatle_calendar = pd.read_csv('../data/Seatle/calendar.csv')

#### Boston data

In [3]:
boston_listings = pd.read_csv('../data/Boston/listings.csv')
boston_reviews = pd.read_csv('../data/Boston/reviews.csv', parse_dates=[2])
boston_calendar = pd.read_csv('../data/Boston/calendar.csv')

### 4. Cleaning data

#### Listings dataset

Vemos que nuestro dataset contiene una gran cantidad de columnas que podrìan ser de utilidad para nuestro anàlisis. Empezando por una limpieza bàsica basta por ahora.

In [4]:
print(f'Seatle listings dataset have {seatle_listings.shape[1]} columns with {seatle_listings.shape[0]} rows')
print(f'Boston listings dataset have {boston_listings.shape[1]} columns with {boston_listings.shape[0]} rows')

Seatle listings dataset have 92 columns with 3818 rows
Boston listings dataset have 95 columns with 3585 rows


In [5]:
# Different columns in both datasets
set(list(seatle_listings.columns)).symmetric_difference(set(list(boston_listings.columns)))

{'access', 'house_rules', 'interaction'}

Empezando con la limpieza de datos, podemos analizar la cantidad de valores nulos en la columna que es de màs interes en este anàlisis.

There are only 2 rows with misssing data in column `host_is_superhost`, so its safe to delete those rows.

In [6]:
seatle_listings['host_is_superhost'].isnull().sum()

2

In [7]:
boston_listings['host_is_superhost'].isnull().sum()

0

In [8]:
seatle_listings = seatle_listings.dropna(subset=['host_is_superhost'])
boston_listings = boston_listings.dropna(subset=['host_is_superhost'])

Dropping columns that have more than 75% of missing data

In [9]:
# Seatle
columns_to_drop_s = list(seatle_listings.columns[seatle_listings.isnull().mean() > 0.75])
seatle_listings.drop(columns=columns_to_drop_s, inplace=True)

# Boston
columns_to_drop_b = list(boston_listings.columns[boston_listings.isnull().mean() > 0.75])
boston_listings.drop(columns=columns_to_drop_b, inplace=True)

Adicionalmente, existen algunas columnas que no son de interes para nuestro análisis. Por lo que de primer paso, podemos eliminar esas columnas.

In [10]:
# Seatle
seatle_listings.drop(columns=['listing_url', 'scrape_id', 'last_scraped', 'host_id', 'notes', 'picture_url', 'xl_picture_url', 'host_url', 'host_about', 'access', 'house_rules', 'interaction'],
                     inplace=True,
                     errors='ignore')

# Boston
boston_listings.drop(columns=['listing_url', 'scrape_id', 'last_scraped', 'host_id', 'notes', 'picture_url', 'xl_picture_url', 'host_url', 'host_about', 'access', 'house_rules', 'interaction'],
                     inplace=True,
                     errors='ignore')

De igual forma, vemos que nuestra columna de interes es de typo `object`, por lo que hay que convertirlo en booleano.

In [11]:
seatle_listings['host_is_superhost'] = seatle_listings['host_is_superhost'].map({'t': True, 'f': False})
boston_listings['host_is_superhost'] = boston_listings['host_is_superhost'].map({'t': True, 'f': False})

Debido a que nuetro anàlisis implica el precio de los Airbnb, es necesario limpiar la columna ya que es de tipo `object` con el inicio de signo de dolares `$`.

In [12]:
seatle_listings['price'] = seatle_listings['price'].replace('[\$\,\.]', '', regex=True).astype(int)
boston_listings['price'] = boston_listings['price'].replace('[\$\,\.]', '', regex=True).astype(int)

#### Reviews dataset

No hay necesidad de eliminar columnas o rows en el dataset de reviews

In [13]:
# Seatle
seatle_reviews.isnull().sum()

listing_id        0
id                0
date              0
reviewer_id       0
reviewer_name     0
comments         18
dtype: int64

In [14]:
# Boston
boston_reviews.isnull().sum()

listing_id        0
id                0
date              0
reviewer_id       0
reviewer_name     0
comments         53
dtype: int64

#### Calendar dataset

El dataset de calendar tampoco es necesario eliminar datos, sin embargo, debido a que los datos se pueden tratar como series de tiempo, es interesante saber el precio nulo debido a que es un indicativo de que el airbnb no se encontraba displonible. En su lugar, se crearà una nueva columna que especifique si es nulo o no para no perder informaciòn en caso de que se requiera implementar un modelo de predicciòn de precios.

In [15]:
# Seatle
seatle_calendar.isnull().sum()

listing_id         0
date               0
available          0
price         459028
dtype: int64

In [16]:
# Boston
boston_calendar.isnull().sum()

listing_id         0
date               0
available          0
price         665853
dtype: int64

In [17]:
# Creating new column
seatle_calendar['null_price'] = np.where(seatle_calendar['price'].isnull(), 1, 0)
boston_calendar['null_price'] = np.where(boston_calendar['price'].isnull(), 1, 0)

### 5. Saving data

In [18]:
# Seatle
seatle_listings.to_csv('../data/processed/Seatle/listings.csv', index=False)
seatle_reviews.to_csv('../data/processed/Seatle/reviews.csv', index=False)
seatle_calendar.to_csv('../data/processed/Seatle/calendar.csv', index=False)

In [19]:
# Boston
boston_listings.to_csv('../data/processed/Boston/listings.csv', index=False)
boston_reviews.to_csv('../data/processed/Boston/reviews.csv', index=False)
boston_calendar.to_csv('../data/processed/Boston/calendar.csv', index=False)

Questions
**Cualidades para ser un superhost**
- Cuales son las areas màs costosas y mas baratas de Seatle?
- Què es lo que define el precio y es superhost(correlaciòn). La temporada! utiliar calendar

- En què areas se encuentran los superhosts de Seatle y Boston?
- Què cualidades debe tener la accommodation para ser un superhost?
- Comparaciòn de precios de host y superhosts
- Anàlisis de reviews de los superhosts.
    - comparaciòn de superhosts con hosts normalitos
    - Usar NLP?
